## Nutzung von Language Models (LLMs) mit Python, LangChain & Co.
In diesem Workshop lernst du Schritt für Schritt, wie du Language Models (LLMs) in Python einbindest. Wir nutzen dafür verschiedene Bibliotheken:

- `dotenv` zum Laden von Umgebungsvariablen,
- `pydantic` zum strukturierten Validieren unserer Ausgaben,
- `langchain_openai` für die Integration mit (OpenAI-kompatiblen) Modellen,
- `langchain_ollama` für die Integration mit Ollama,
- und eine kleine interne Bibliothek `langchain_core.messages` für die Message-Klassen HumanMessage, SystemMessage und AIMessage.

### 1. Import

In [1]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# LangChain-spezifische Imports
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

### 2. Laden der Umgebungsvariablen


In [2]:
# Laden der Variablen aus .env (falls vorhanden)
load_dotenv()

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://host.docker.internal:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2:1b")

### 3. Konfiguration für Ollama (lokales LLM)

Hier erstellen wir eine Instanz des Ollama Chat LLMs, einen sogenannten Wrapper (um ein LLM). 

> **_Achtung:_**  Bevor wir Ollama nutzen können, müssen wir zunächst Ollama über einen Docker Container bzw. die `docker-compose.yml` starten. Dass müssen wir außerhalb des DevContainers tun, indem wir `docker compose up` im Verzeichnis des Repos in der Konsole eingeben.

In [3]:
# Erstellen einer Instanz des Ollama-Chat-LLMs
ollama_llm = ChatOllama(
    model=OLLAMA_MODEL,
    base_url=OLLAMA_BASE_URL,
    temperature=0.8
)

#### Erklärung:

- `os.getenv("OLLAMA_BASE_URL", "http://host.docker.internal:11434")` versucht erst den Wert aus den Umgebungsvariablen zu lesen. Falls nichts gefunden wird, nutzen wir den Default-Wert.
- `ChatOllama` ist eine Klasse aus `langchain_ollama`, die uns eine einfache Schnittstelle zum Ollama-Server bietet.
- Wir setzen `temperature=0.8`, was bedeutet, dass die Antworten etwas zufälliger (kreativer) ausfallen als mit dem Standardwert (z. B. 0.7). Je höher die Temperatur, desto "kreativer" bzw. "zufälliger" sind die Antworten.

[Hier](https://python.langchain.com/v0.2/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html) kannst du nachschauen, was du alles bei deinem Modell einstellen kannst.

[Hier](https://ollama.com/library) kannst du nachschauen, was für Modelle noch so von Ollama unterstützt werden.

### 4. Erste Abfrage an dein lokales Ollama-LLM

In [4]:
prompt = 'Why is the sky blue?'

messages_for_llm = [
    # SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=prompt)
]

answer = ollama_llm.invoke(messages_for_llm)
print(answer.content)

The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first explained it in the late 19th century. Here's a simplified explanation:

When sunlight enters Earth's atmosphere, it consists of a spectrum of colors, including all the colors of the visible light spectrum (red, orange, yellow, green, blue, indigo, and violet). These colors are separated by different wavelengths, with shorter wavelengths (like violet) being scattered more than longer wavelengths (like red).

The scattering of sunlight occurs when tiny molecules of gases in the atmosphere, such as nitrogen (N2) and oxygen (O2), interact with the light. The smaller molecules scatter the shorter wavelengths (blue and violet) more than the longer wavelengths (red and orange), which is known as Rayleigh scattering.

As a result, the blue light is scattered throughout the atmosphere, giving the sky its blue appearance. This is especially noticeable during 

#### Erklärung:

- Wir erstellen ein prompt mit der Frage: `"Why is the sky blue?"`
- `messages_for_llm` ist eine Liste, die (mindestens) ein `HumanMessage` enthält. (Optional kann man auch ein `SystemMessage` hinzufügen, um den Kontext/Verhalten des Modells zu steuern.)
- `invoke(...)` ruft tatsächlich das Large Language Model auf und gibt die Antwort zurück.
- Mit `answer.content` greifen wir auf den reinen Text der generierten Antwort zu und geben ihn aus.

### 5. Nutzung von strukturiertem Output mit Pydantic
Das folgende Code-Beispiel zeigt, wie man den Output eines LLM direkt in ein Pydantic-Modell parsen kann. Das ist sehr praktisch, wenn wir z. B. JSON-ähnliche Antworten erwarten oder festes Schema (Felder) haben wollen.

In [5]:
class QuestionAnswer(BaseModel):
    """A model with an answer field"""
    answer_as_string: str = Field(description="The answer generated")

# Vorbereitung unserer Nachrichten
messages_for_llm = [
    HumanMessage(content=prompt)
]

# Wir nutzen .with_structured_output(...) um die Antwort direkt zu validieren
structured_llm = ollama_llm.with_structured_output(QuestionAnswer)
answer_structured = structured_llm.invoke(messages_for_llm)
print(answer_structured.answer_as_string)


The sky appears blue because of a phenomenon called scattering, where light from the sun is broken down into its individual colors (red, orange, yellow, green, blue, and violet) by the tiny molecules of gases in the Earth's atmosphere.


#### Erklärung:

- `QuestionAnswer` ist ein Pydantic-Datenmodell. Es definiert, wie die Antwort von LLM aussehen soll – in diesem Fall mit dem Feld `answer_as_string`.
- `with_structured_output(QuestionAnswer)` instruiert LangChain, dass die Antwort vom LLM dem Schema `QuestionAnswer` entsprechen soll (intern wird das Prompting entsprechend angepasst und versucht, die Antwort im passenden JSON-Format zurückzubekommen).
- `answer_structured.answer_as_string` liefert schließlich den generierten Text zurück.

### 6. Alternative: StackIT LLM mit OpenAI-kompatiblem Endpoint
Nun wollen wir noch ein anderes LLM ansprechen – z. B. über **StackIT**. Dieses funktioniert mit einem OpenAI-kompatiblen Endpoint und kann daher über ChatOpenAI (aus langchain_openai) verwendet werden.

Dazu müssen wir zunächst den API_KEY in die `.env` oder in der Code Zeile darunter einsetzen. 

Der Vorteil dieses Modells ist, dass es im Gegensatz zu unserem lokalen LLM viel mehr Parameter hat (70 Milliarden) und damit bessere Ergebnisse liefert. Dafür läuft es auf 4 L40S Nvidia GPUs. 

In [6]:

# Wir lesen die nötigen Umgebungsvariablen ein
STACKIT_VLLM_API_KEY = os.getenv("STACKIT_VLLM_API_KEY", "")
STACKIT_MODEL = os.getenv("STACKIT_MODEL", "neuralmagic/Meta-Llama-3.1-70B-Instruct-FP8")
STACKIT_BASE_URL = os.getenv("STACKIT_BASE_URL", "https://api.openai-compat.model-serving.eu01.onstackit.cloud/v1")

# Erstellen der ChatOpenAI-Instanz
stackit_llm = ChatOpenAI(
    model=STACKIT_MODEL,
    base_url=STACKIT_BASE_URL,
    api_key=STACKIT_VLLM_API_KEY,
)

prompt = "Why is the sky blue?"

messages_for_stackit = [HumanMessage(content=prompt)]
structured_llm_stackit = stackit_llm.with_structured_output(QuestionAnswer)
answer_stackit = structured_llm_stackit.invoke(messages_for_stackit)

print(answer_stackit)

answer_as_string='The sky appears blue because of a phenomenon called Rayleigh scattering, which is the scattering of sunlight by small particles or molecules in the atmosphere. The blue color we see in the sky is due to the scattering of shorter (blue) wavelengths of light by the tiny molecules of gases such as nitrogen and oxygen, while the longer (red) wavelengths pass straight through. This scattering effect gives the sky its blue appearance during the daytime.'


#### Erklärung:

- `ChatOpenAI(...)` nimmt ähnliche Parameter wie die offiziellen OpenAI-Clients. Wir müssen lediglich base_url und api_key anpassen, um den Service von StackIT anzusprechen.
- Auch hier nutzen wir wieder `with_structured_output(QuestionAnswer)`, um das Antwortformat festzulegen.
- Mit `print(answer_stackit)` bekommen wir die Daten entsprechend unserem Pydantic-Modell zurück (in diesem Fall enthält das Objekt ein Feld answer_as_string).

## Zusammenfassung
In diesem Workshop hast du gelernt:

1. Wie man Umgebungsvariablen einsetzt, um sensible Daten (z. B. API-Keys, Endpunkte) zu trennen und flexibel zu halten.
2. Wie man mit LangChain unterschiedliche LLMs anspricht, sowohl Ollama als auch OpenAI-kompatible Modelle.
3. Wie man mittels Pydantic und der LangChain-Methode with_structured_output() Antworten von LLMs direkt in ein festes Schema gießen kann, um sie leicht weiterzuverarbeiten.

Die gezeigten Codebeispiele sollten dich in die Lage versetzen, eigene LLM-basierte Anwendungen oder Chatbots zu entwickeln. Die wichtigsten Prinzipien sind dabei:
- Prompting (welche Eingaben/Anweisungen gebe ich an das Modell?),
- Nachrichtenformat (System-/Human-/AI-Nachrichten in langchain_core.messages),
- Strukturierte Validierung (z. B. mit Pydantic).

Viel Erfolg bei deinen Experimenten mit Language Models!

### Weiterführende Ideen oder Challenge:

- Erweitere das Pydantic Modell um weitere Ausgabefelder (bspw. eine Liste) und versuche durch die Validierung des Pydantic Parsers zu "bestehen".  
- Binde das System in ein FastAPI-Backend ein und nutze die strukturierten Antworten, um JSON-Antworten an einen Frontend-Client zu liefern.
- Experimentiere mit unterschiedlichen Temperature-Einstellungen und Prompting-Techniken (z. B. few-shot, chain-of-thought).
